In [10]:
from pymongo import MongoClient

client = MongoClient()
db = client.StarTrek_Database
col = db.scripts

In [8]:
# import json
# with open('Data/st_scripts.json') as data_file:    
#     data = json.load(data_file)

In [14]:
# for episode in data:
#     col.insert_one(episode)

In [ ]:
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> averaged_perceptron_tagger
        /home/aleksod/nltk_data...
      Unzipping taggers/averaged_perceptron_tagger.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> maxent_treebank_pos_tagger
Command 'maxent_treebank_pos_tagger' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> u

Nothing to update.


  [P] book................ Everything used in the NLTK Book
  [P] popular............. Popular packages
  [ ] third-party......... Third-party data packages

([*] marks installed packages; [P] marks partially installed collections)

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> 

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> maxent_treebank_pos_tagger
        /home/aleksod/nltk_data...
      Unzipping taggers/maxent_treebank_pos_tagger.zip.

---------------------------------------------------------------------------
    d) Download   l) List    u) Update  

In [24]:
text = '''
Screenplay by:\n                            GENE RODDENBERRY &  HAROLD LIVINGSTONE\n\n                                           Story by:\n                             ALAN DEAN FOSTER  & GENE RODDENBERRY\n\n                   \n                                                                  SHOOTING SCRIPT\n\n                                                                    July 19, 1978\n\n\n\n                   \n\n                   FADE IN :\n\n               1   EXT SPACE (S)                                                1\n\n                   An ever expanding infinity of light and color as CAMERA                \n                   TRAVELS THROUGH deep space, MOVING DIRECTLY for one\n                   pinpoint of light: a STAR GROWING RAPIDLY as we SWEEP\n                   TOWARD IT, a normal white star SUDDENLY CHANGING,\n                   brightening, flaring unbelievable intensity: supernova.\n
'''

In [25]:
sentences = sent_tokenize(text)
print(sentences)

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/PY3/english.pickle' not found.
  Please use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/aleksod/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************